## Step 1: Setup & Imports

In [ ]:
from pathlib import Path
import random
from ultralytics.models import YOLO
import torch
import yaml

random.seed(42)

## Step 2: Configure Paths & Hyperparameters

In [ ]:
# Dataset path (created by e2e_data_prep.ipynb)
YOLO_DATASET = Path("datasets/ready/full_dataset")
RUNS_DIR = Path("runs/segment")

# Verify dataset exists
if not YOLO_DATASET.exists():
    raise FileNotFoundError(f"Dataset not found at {YOLO_DATASET}. Run e2e_data_prep.ipynb first!")

print(f"Dataset: {YOLO_DATASET}")
print(f"  Train: {YOLO_DATASET / 'train'}")
print(f"  Val: {YOLO_DATASET / 'val'}")
print(f"  Test: {YOLO_DATASET / 'test'}")

In [ ]:
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 640
model_type = "yolo11n-seg.pt"
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f"Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA: {torch.version.cuda}")

In [ ]:
AUG_CONFIG = {
    'hsv_h': 0.015,  # Hue augmentation
    'hsv_s': 0.7,    # Saturation
    'hsv_v': 0.4,    # Value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,     # Scaling
    'shear': 0.0,     # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.0,    # Vertical flip
    'fliplr': 0.5,    # Horizontal flip
    'mosaic': 1.0,    # Mosaic augmentation
    'mixup': 0.0,     # Mixup augmentation
}

## Step 3: Verify Dataset Structure

Dataset is already prepared by e2e_data_prep.ipynb

In [ ]:
# Verify dataset structure
print("="*60)
print("DATASET VERIFICATION")
print("="*60)

splits = ['train', 'val', 'test']
stats = {}

for split in splits:
    img_dir = YOLO_DATASET / split / "images"
    lbl_dir = YOLO_DATASET / split / "labels"
    
    if img_dir.exists() and lbl_dir.exists():
        num_images = len(list(img_dir.glob("*")))
        num_labels = len(list(lbl_dir.glob("*.txt")))
        stats[split] = {'images': num_images, 'labels': num_labels}
        print(f"{split.upper():5s}: {num_images:4d} images, {num_labels:4d} labels")
    else:
        stats[split] = {'images': 0, 'labels': 0}
        print(f"{split.upper():5s}: Missing!")

total_images = sum(s['images'] for s in stats.values())
total_labels = sum(s['labels'] for s in stats.values())

print(f"{'TOTAL':5s}: {total_images:4d} images, {total_labels:4d} labels")
print("="*60)

if total_images == 0:
    raise RuntimeError("No dataset found! Run e2e_data_prep.ipynb to create the dataset.")

## Step 4: Create YOLO Configuration File

In [ ]:
classes = {
    'red ball': 0,
    'human': 1,
    'trashcan': 2
}

config = {
    'path': str(YOLO_DATASET.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(classes),
    'names': list(classes.keys())
}

config_path = YOLO_DATASET / 'data.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✓ Configuration saved: {config_path}")
print(f"\nDataset structure:")
print(f"  Train: {YOLO_DATASET / 'train'}")
print(f"  Val: {YOLO_DATASET / 'val'}")
print(f"  Test: {YOLO_DATASET / 'test'}")

## Step 5: Train Model

Train YOLOv11 with:
- Data augmentation on train set
- Checkpoints saved for best model
- Validation after each epoch

In [ ]:
# Load pretrained model
model = YOLO(model_type)

In [ ]:
head_idx = next((i for i, m in enumerate(model.model.model) if 'Detect' in m.__class__.__name__ or 'Segment' in m.__class__.__name__), len(model.model.model) - 1)
project_name = 'ball_person_model'
results = model.train(
    data=str(config_path),
    epochs=500,
    freeze=list(range(head_idx)),
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE,
    project=str(RUNS_DIR),
    name=project_name,
    exist_ok=True,
    
    # Checkpointing
    save=True,
    save_period=1,  # Save every epoch
    
    # Validation
    val=True,
    
    # Data augmentation (only applied to train)
    **AUG_CONFIG,
    
    # Optimizer
    optimizer='Adam',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Loss weights
    box=7.5,
    cls=0.5,
    dfl=1.5,
    
    # Other
    patience=20,  # Early stopping
    workers=8,

    verbose=True)

## Step 6: Evaluate Results

In [ ]:
# Validation metrics
metrics = model.val()

print("\n" + "="*60)
print("📊 VALIDATION METRICS")
print("="*60)
print(f"Box mAP50: {metrics.box.map50:.4f}")
print(f"Box mAP50-95: {metrics.box.map:.4f}")
print(f"Mask mAP50: {metrics.seg.map50:.4f}")
print(f"Mask mAP50-95: {metrics.seg.map:.4f}")

In [ ]:

# Find best checkpoint
model_dir = RUNS_DIR / 'ball_person_model'
best_model = model_dir / 'weights' / 'best.pt'
last_model = model_dir / 'weights' / 'last.pt'

print(f"\n✅ Training complete!")
print(f"   Best model: {best_model}")
print(f"   Last model: {last_model}")
print(f"   Results: {model_dir}")

## Step 7: Test on Sample Images (Optional)

In [ ]:
RUNS_DIR

In [ ]:
# Load best model
best_model_path = RUNS_DIR / 'ball_person_model' / 'weights' / 'last.pt'
trained_model = YOLO(str(best_model_path))

# Test on validation images (sample from val set)
test_images = list((YOLO_DATASET / "val" / "images").glob("*"))

print(f"Testing on {len(test_images)} sample images...")

for img_path in test_images:
    results = trained_model.predict(str(img_path), save=True, conf=0.25)
    print(f"  ✓ {img_path.name}")

print(f"\nResults saved to: {RUNS_DIR / 'ball_person_model'}")